# Energy Load Forecasting — Model Comparison

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PhysicsInforMe/scientific-prototypes/blob/main/energy-load-forecasting-benchmark/notebooks/02_model_comparison.ipynb)

This notebook runs the full benchmark comparing **Time Series Foundation Models** (Chronos-Bolt, Chronos-2) against statistical baselines (Seasonal Naive, SARIMA) on ERCOT hourly load data.

## 0. Setup (Colab)

Uncomment and run the cell below when using Google Colab.

In [ ]:
# # --- Colab setup ---
# !git clone https://github.com/PhysicsInforMe/scientific-prototypes.git
# %cd scientific-prototypes/energy-load-forecasting-benchmark
# !pip install -q -e ".[models]"
#
# # Verify GPU
# import torch
# print(f"GPU available: {torch.cuda.is_available()}")
# if torch.cuda.is_available():
#     print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Load ERCOT Data

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

from energy_benchmark.data import ERCOTLoader
from energy_benchmark.data.preprocessing import preprocess_series

loader = ERCOTLoader(years=[2020, 2021, 2022, 2023, 2024])
series = loader.load()
series = preprocess_series(series)

train, val, test = loader.split(series)
print(f"Train: {len(train):,} hrs | Val: {len(val):,} hrs | Test: {len(test):,} hrs")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(14, 4))
train.plot(ax=ax, label="Train", alpha=0.7)
val.plot(ax=ax, label="Validation", alpha=0.7)
test.plot(ax=ax, label="Test", alpha=0.7)
ax.set_ylabel("Load (MW)")
ax.set_title("ERCOT Hourly Load — Train / Val / Test Split")
ax.legend()
plt.tight_layout()
plt.show()

## 2. Initialize Models

In [ ]:
import torch
from energy_benchmark.models import SeasonalNaiveModel

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

models = []

# --- Baselines ---
models.append(SeasonalNaiveModel(seasonality=168))

# --- Foundation Models ---
try:
    from energy_benchmark.models import ChronosBoltModel
    models.append(ChronosBoltModel(model_size="base", device=device))
except ImportError:
    print("chronos-forecasting not installed, skipping Chronos-Bolt")

try:
    from energy_benchmark.models import Chronos2Model
    models.append(Chronos2Model(device=device))
except ImportError:
    print("chronos-forecasting not installed, skipping Chronos-2")

print(f"Models to evaluate: {[m.name for m in models]}")

In [ ]:
# Fit all models (loads weights for foundation models)
for m in models:
    print(f"Fitting {m.name}...")
    m.fit(train)
    print(f"  {m.name} ready.")

## 3. Run Benchmark

In [ ]:
from energy_benchmark.evaluation import BenchmarkRunner

runner = BenchmarkRunner(
    models=models,
    prediction_horizons=[24, 168],
    context_lengths=[512],
    num_samples=100,
    metric_names=["mae", "rmse", "mase"],
)

results = runner.run(
    train, test,
    rolling_config={"step_size": 24, "num_windows": 30},
)

df = results.to_dataframe()
df

## 4. Results Visualization

In [ ]:
from energy_benchmark.visualization import plot_comparison, plot_metric_heatmap

for metric in ["mae", "mase"]:
    if metric in df.columns:
        plot_comparison(df, metric=metric)
        plt.show()

for metric in ["mae", "mase"]:
    if metric in df.columns:
        plot_metric_heatmap(df, metric=metric)
        plt.show()

## 5. Example Forecast

In [ ]:
from energy_benchmark.visualization import plot_probabilistic_forecast
import numpy as np

# Pick the first forecast window from the first model with samples
for fr in results.forecasts:
    if fr.samples is not None:
        plot_probabilistic_forecast(
            actual=fr.actual,
            point_forecast=fr.point_forecast,
            samples=fr.samples,
            title=f"{fr.model_name} — {fr.horizon}h forecast (window {fr.window_idx})",
        )
        plt.show()
        break
else:
    print("No probabilistic forecasts available.")

## 6. Save Results

In [ ]:
from pathlib import Path

out_dir = Path("../results/tables")
out_dir.mkdir(parents=True, exist_ok=True)
df.to_csv(out_dir / "benchmark_results.csv", index=False)
print(f"Results saved to {out_dir / 'benchmark_results.csv'}")